In [1]:
import utils.common
import evaluation
import importlib
import numpy as np
import time
import models.MatrixFactorization
import models.BPR
import models.ConstraintAutoRec
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.MatrixFactorization)
importlib.reload(models.BPR)

<module 'models.BPR' from '/home/ec2-user/SageMaker/rs-ml-with-constraints/models/BPR.py'>

In [2]:
ev = evaluation.Evaluation(utils.common.ml_small)

In [3]:
model = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.ml_small['dimensions'], epochs=10, novelty_weight=0.1, diversity_weight=0.1, latent_dim=64)
model.train(utils.common.load_dataset(utils.common.ml_small, 'train'), utils.common.ml_small['train']['records'])

Epoch 1/10
19/19 [==============================] - 15s 791ms/step - loss: 0.2561 - accuracy: 0.0000e+00
Epoch 2/10
19/19 [==============================] - 0s 18ms/step - loss: 0.1787 - accuracy: 0.0164
Epoch 3/10
19/19 [==============================] - 0s 24ms/step - loss: 0.1581 - accuracy: 0.0474
Epoch 4/10
19/19 [==============================] - 0s 17ms/step - loss: 0.1593 - accuracy: 0.0312
Epoch 5/10
19/19 [==============================] - 0s 17ms/step - loss: 0.1533 - accuracy: 0.0073
Epoch 6/10
19/19 [==============================] - 0s 18ms/step - loss: 0.1453 - accuracy: 0.0296
Epoch 7/10
19/19 [==============================] - 0s 18ms/step - loss: 0.1401 - accuracy: 0.0312
Epoch 8/10
19/19 [==============================] - 0s 18ms/step - loss: 0.1392 - accuracy: 0.0251
Epoch 9/10
19/19 [==============================] - 0s 18ms/step - loss: 0.1216 - accuracy: 0.0237
Epoch 10/10
19/19 [==============================] - 0s 17ms/step - loss: 0.1189 - accuracy: 0.0230


In [4]:
ev.evaluate_single_thread(model)

Batch nr 1 predicted
Batch nr 2 predicted


{'accuracy': 0.7226011197618034,
 'precision': 0.8949401562824231,
 'recall': 0.764382022251553,
 'map@1': 0.13513513513513514,
 'map@5': 0.06660285285285285,
 'map@10': 0.04873461258877926,
 'diversity@5': 0.21513328565133855,
 'diversity@10': 0.2533638369443251,
 'epc@5': 0.7152287925633201,
 'epc@10': 0.7328247190794073,
 'epd@5': 0.3091634445805061,
 'name': 'ConstraintAutoRec',
 'dimensions': 10381,
 'latent_dims': 64,
 'accuracy_weight': 1.0,
 'novelty_weight': 0.1,
 'diversity_weight': 0.1,
 'epochs': 10,
 'batch_size': 32,
 'optimizer': 'adam'}

In [ ]:
diversity_model = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=30, novelty_weight=0.25, diversity_weight=2.0)
diversity_model.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])


In [ ]:
def create_model(novelty_weight):
    m = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=15, novelty_weight=novelty_weight, diversity_weight=0, batch_size=128)
    m.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])
    return m

nov_models2 = [create_model(x) for x in [0.1, 0.5, 1.0, 2.0]]

In [ ]:
import matplotlib.pyplot as plt
models

In [ ]:
#ev = evaluation.Evaluation(utils.common.movie_lens)
#big_eval = ev.evaluate_models(models)
big_eval.head()

In [ ]:
plt.bar(eval_data.index, eval_data['epc@10'])

In [ ]:
plt.plot(eval_data['novelty_weight'], eval_data['epc@10'], 'ro', ls='--')
plt.plot(eval_data['novelty_weight'], eval_data['accuracy'], 'bo', ls='--')
plt.show()

In [ ]:
def plot_bar(data, axis, x_param):
    bottom = data[axis].min() 
    top = data[axis].max()
    margin = (top - bottom) * 0.05
    fig, ax = plt.subplots()
    ax.bar(data.index, data[axis] )
    ax.set_xticks(data.index)
    ax.set_xticklabels(data[x_param])
    ax.set_ylabel(axis)
    ax.set_xlabel(x_param)
    plt.show()

In [3]:
# mf = models.MatrixFactorization.MatrixFactorization(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=10, latent_dim = 64)
# mf.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])

bpr = models.BPR.BPR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=5, latent_dim = 32, batch_size=1024)

In [209]:
bpr.batch_size = 8092
bpr.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])

Epoch #0 Loss at step 1610: 0.2968
Epoch #1 Loss at step 1610: 0.2310
Epoch #2 Loss at step 1610: 0.1927
Epoch #3 Loss at step 1610: 0.1764
Epoch #4 Loss at step 1610: 0.1552


In [4]:
result = ev.evaluate(bpr)

Batch nr 1 predicted
Batch nr 2 predicted
Batch nr 3 predicted
Batch nr 4 predicted
Batch nr 5 predicted
Batch nr 6 predicted
Batch nr 7 predicted
Batch nr 8 predicted
Batch nr 9 predicted
Batch nr 10 predicted
Batch nr 11 predicted
Batch nr 12 predicted
Batch nr 13 predicted
Batch nr 14 predicted
Batch nr 15 predicted
Batch nr 16 predicted
Batch nr 17 predicted
Batch nr 18 predicted
Batch nr 19 predicted
Batch nr 20 predicted
Batch nr 21 predicted
Batch nr 22 predicted
Batch nr 23 predicted
Batch nr 24 predicted
Batch nr 25 predicted
Batch nr 26 predicted
Batch nr 27 predicted
Batch nr 28 predicted
Batch nr 29 predicted
Batch nr 30 predicted
Batch nr 31 predicted
Batch nr 32 predicted
Batch nr 33 predicted
Batch nr 34 predicted
Batch nr 35 predicted
Batch nr 36 predicted
Batch nr 37 predicted
Batch nr 38 predicted
Batch nr 39 predicted
Batch nr 40 predicted
Batch nr 41 predicted
Batch nr 42 predicted
Batch nr 43 predicted
Batch nr 44 predicted
Batch nr 45 predicted
Batch nr 46 predict

In [5]:
result

{'accuracy': 0.24272472285478736,
 'precision': 0.5270423815721148,
 'recall': 0.20473174005475447,
 'map@1': 0.0009856892523364485,
 'map@5': 0.0005072141906801659,
 'map@10': 0.0003713081562453642,
 'diversity@5': 0.24208009880360445,
 'diversity@10': 0.24206584267840847,
 'epc@5': 0.6275318828936683,
 'epc@10': 0.6274940655597349,
 'epd@5': 0.24108397364521375,
 'name': 'BPR',
 'latent_dim': 32,
 'epochs': 5,
 'batch_size': 1024}

In [127]:
result = ev.evaluate_models([model, bpr])

Evaluating ConstraintAutoRec |============================================================>| 100.0% 
Evaluating BPR |============================================================>| 100.0% 


In [ ]:
ev.evaluate_models(nov_models2)

In [211]:
result

{'accuracy': 0.4768148516836452,
 'precision': 0.5390853520053073,
 'recall': 0.5404514802406271,
 'map@1': 0.09876236497147713,
 'map@5': 0.055156317693372335,
 'map@10': 0.043819520386634364,
 'diversity@5': 0.16117022541419124,
 'diversity@10': 0.16641964375980106,
 'epc@5': 0.4180729404503865,
 'epc@10': 0.42959490156137525,
 'epd@5': 0.2049803661056044,
 'name': 'BPR',
 'latent_dim': 32,
 'epochs': 5,
 'batch_size': 8092}